# B5W3: End-to-End Insurance Risk Analytics & Predictive Modeling
## Task 4
    - Build and evaluate predictive models

In [7]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, roc_auc_score, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt
import shap 
from sklearn.impute import SimpleImputer

In [8]:
sys.path.append(os.path.abspath(os.path.join('..')))

In [9]:
df  = pd.read_csv('../data/MachineLearningRating_v3.csv', low_memory=False )

#### Missing values

In [5]:
df.isnull().sum()

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                         145961
AccountType                   40232
MaritalStatus                  8259
Gender                         9536
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                       552
cubiccapacity                   552
kilowatts                       552
bodytype                        552
NumberOfDoors               

In [10]:
# --- Strategy Implementation ---

# 1. Identify and drop columns with very high missingness (> 60%)
columns_to_drop = [
    'NumberOfVehiclesInFleet',
    'CrossBorder',
    'CustomValueEstimate',
    'WrittenOff',
    'Rebuilt',
    'Converted'
]
df_cleaned = df.drop(columns=columns_to_drop)
print(f"Dropped {len(columns_to_drop)} columns with high missing data.")

# 2. Drop rows with low missingness (< 5%)
# This is a safe and clean approach for columns like Gender, MaritalStatus, and vehicle details
# We'll drop rows where any of these columns have a missing value.
rows_to_drop_na = [
    'Gender',
    'MaritalStatus',
    'CapitalOutstanding',
    'mmcode', 'VehicleType', 'make', 'Model', 'Cylinders', 'cubiccapacity',
    'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate'
]

# Create a copy to avoid SettingWithCopyWarning
df_cleaned = df_cleaned.dropna(subset=rows_to_drop_na).copy()
print(f"Dropped rows with missing values in key low-missingness columns.")
print(f"Remaining rows after dropping: {df_cleaned.shape[0]}")

# 3. Impute columns with moderate missingness (5-30%)
# These columns are `Bank`, `AccountType`, and `NewVehicle`
# Let's check their data types to choose the right imputation strategy
# `Bank` and `AccountType` are likely categorical. `NewVehicle` is likely binary.
# We will use the `most_frequent` (mode) strategy.

imputation_features = ['Bank', 'AccountType', 'NewVehicle']
imputer = SimpleImputer(strategy='most_frequent')

# The imputer needs to be fitted on the data
df_cleaned[imputation_features] = imputer.fit_transform(df_cleaned[imputation_features])
print("Imputed missing values in 'Bank', 'AccountType', and 'NewVehicle' with the mode.")

# Final check of missing values after cleaning
print("\nFinal Missing Value Summary:")
print(df_cleaned.isnull().sum().sort_values(ascending=False))

# Update your `modeling_features` list to reflect the changes
modeling_features = [
    'Province', 'Gender', 'make', 'body_type', 'Cylinders', 'VehicleAge',
    'CapitalOutstanding', 'SumInsured', 'CalculatedPremiumPerTerm',
    'Bank', 'AccountType', 'NewVehicle', 'MaritalStatus' # Add imputed features
]
# Note: 'Model' is a high-cardinality feature. We will handle it with one-hot encoding later.
# You might need to add `cubiccapacity`, `kilowatts`, and `NumberOfDoors` if you want to use them.

Dropped 6 columns with high missing data.
Dropped rows with missing values in key low-missingness columns.
Remaining rows after dropping: 989557
Imputed missing values in 'Bank', 'AccountType', and 'NewVehicle' with the mode.

Final Missing Value Summary:
UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity           